In [21]:
import carla
import math
import random
import time
import queue
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
#  Trying to import the Global and Local 
sys.path.append(("C:/CARLA/WindowsNoEditor/PythonAPI/carla"))
from agents.navigation.global_route_planner import GlobalRoutePlanner

client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
world  = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

actor_lst = []

# spawn vehicle
start_transform = random.choice(spawn_points)
end_transform = random.choice(spawn_points)

vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, start_transform)

vehicle.set_autopilot(True)
actor_lst.append(vehicle)


#spawn traffic
traff_bp = bp_lib.filter('vehicle.*')
for i in range(50):
    traf = world.try_spawn_actor(random.choice(traff_bp), random.choice(spawn_points))
    if traf:
        actor_lst.append(traf)
        traf.set_autopilot(True)

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-5,z=20)), carla.Rotation(yaw=90, pitch=-90)) 
spectator.set_transform(transform)

# spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
# camera_init_trans = carla.Transform(carla.Location(z=2))
camera_init_trans =carla.Transform(vehicle.get_transform().transform(carla.Location(x=-5,z=20)), carla.Rotation(yaw=90, pitch=-90))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
vehicle.set_autopilot(True)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get the map spawn points
spawn_points = world.get_map().get_spawn_points()

# Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [22]:
'''Tracing a route between the two specified point '''

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
route =  grp.trace_route(start_transform.location, end_transform.location)

''' This is just to visualize the route'''
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)
    
# print("Done")

waypoint_lst = []
i = 0
for waypoint, _ in route:
    i+=1
    loc = waypoint.transform.location
    waypoint_lst.append((loc)) 
    
print(len(waypoint_lst),i)
    # print(asadbhai)

311 311


In [23]:

def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

# Get the world to camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

In [24]:



# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

In [25]:
# Set up the set of bounding boxes from the level
# We filter for traffic lights and traffic signs




# Remember the edge pairs
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]
# Retrieve the first image
world.tick()
image = image_queue.get()

# Reshape the raw data into an RGB array
img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

# Display the image in an OpenCV display window
cv2.namedWindow('ImageWindowName', cv2.WINDOW_AUTOSIZE)
cv2.imshow('ImageWindowName',img)
cv2.waitKey(1)
while True:
    # Retrieve and reshape the image
    world.tick()
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    # Get the camera matrix 
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    # print(world_2_camera/)
    bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.Vehicles)
    bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.Vehicles))
    for bb in bounding_box_set:

        # Filter for distance from ego vehicle
        if bb.location.distance(vehicle.get_transform().location) < 50:

            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the bounding box. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
            forward_vec = vehicle.get_transform().get_forward_vector()
            ray = bb.location - vehicle.get_transform().location
            
            # if forward_vec.x *ray.x+forward_vec.y *ray.y+forward_vec.z *ray.z > 1:
                # Cycle through the vertices
            verts = [v for v in bb.get_world_vertices(carla.Transform())]
            for edge in edges:
                # Join the vertices into edges
                p1 = get_image_point(verts[edge[0]], K, world_2_camera)
                p2 = get_image_point(verts[edge[1]],  K, world_2_camera)
                # Draw the edges into the camera output
                cv2.line(img, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), (0,0,255, 255), 1)
        print(len(waypoint_lst))
        for way in waypoint_lst:
            w = get_image_point(way, K, world_2_camera)
            
            cv2.circle(img, (int(w[0]),int(w[1])),5,(255, 0, 0), 2)
    # Now draw the image into the OpenCV display window
    cv2.imshow('ImageWindowName',img)
    
    # Break the loop if the user presses the Q key
    if cv2.waitKey(1) == ord('q'):
        break

# Close the OpenCV display window when the game loop stops
# cv2.destroyAllWindows()

311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311
311


In [26]:
# vehicle.destroy()
''' Destroy everything'''
try:
    car_lst = world.get_actors().filter('vehicle.*')
    print(car_lst)
    if car_lst:
        for car in car_lst:
            car.destroy()
        world.tick()
except:
    print("Nope doesnt work")

[Actor(id=770, type=vehicle.bh.crossbike), Actor(id=771, type=vehicle.lincoln.mkz_2020), Actor(id=768, type=vehicle.bmw.grandtourer), Actor(id=741, type=vehicle.lincoln.mkz_2020), Actor(id=767, type=vehicle.tesla.model3), Actor(id=764, type=vehicle.harley-davidson.low_rider), Actor(id=763, type=vehicle.ford.ambulance), Actor(id=762, type=vehicle.nissan.micra), Actor(id=761, type=vehicle.diamondback.century), Actor(id=760, type=vehicle.bh.crossbike), Actor(id=769, type=vehicle.bmw.grandtourer), Actor(id=759, type=vehicle.mercedes.coupe_2020), Actor(id=766, type=vehicle.jeep.wrangler_rubicon), Actor(id=765, type=vehicle.kawasaki.ninja), Actor(id=756, type=vehicle.lincoln.mkz_2017), Actor(id=755, type=vehicle.toyota.prius), Actor(id=754, type=vehicle.dodge.charger_police), Actor(id=753, type=vehicle.audi.etron), Actor(id=752, type=vehicle.bmw.grandtourer), Actor(id=751, type=vehicle.dodge.charger_police_2020), Actor(id=750, type=vehicle.toyota.prius), Actor(id=749, type=vehicle.volkswagen

In [27]:
cv2.__version__

'4.7.0'